# **CellTracksColab - Distance to ROI**
---
<font size = 4> This notebook is specifically designed to analyze movement tracks in relation to designated Regions of Interest (ROIs). Its primary aim is to compute and analyze the distances between moving objects (tracks) and ROIs, which may also be dynamic. By evaluating these distances over time, the notebook provides insights into the spatial behavior of the tracked entities relative to the ROIs. The notebook integrates data processing, distance calculations, and statistical analysis to offer a comprehensive tool for researchers to study movement patterns, interactions, and the overall dynamics of moving entities in relation to critical areas of interest. Before beginning, ensure that your data is properly loaded in the CellTracksColab format for optimal analysis.

---

# **Part 0. Before getting started**
---

<font size = 5>**Important notes**

---

<font size = 5>**Data Requirements for Analysis**

<font size = 4>Be advised of one significant limitation inherent to this notebook.

<font size = 4 color="red">**This notebook only supports 2D + t datasets**</font>.

---
<font size = 5>**Prerequisites for Using This Notebook**

<font size = 4>To effectively utilize this notebook for analyzing track distances to Regions of Interest (ROIs), the following prerequisites are essential:
<font size = 4>
1. **DataFrames from CellTrackColab**:
   - Ensure you have the `spots` and `tracks` DataFrames compiled by CellTrackColab.
<font size = 4>
2. **ROI Images in TIF Format**:
   - The ROI images or movies should be in TIF (`.tif`) file format. These images should be mask or label images where the background has a pixel value of 0.
<font size = 4>
3. **Proper Naming of ROI Images**:
   - Adopt a consistent and unique naming convention for your ROIs. This naming should be reflected in both your data analysis and image files.
   - Follow the file naming format: `File_name_ROI_name.tif`. For example, if your tracking file is named 'sample' and the ROI is 'nuclei', the corresponding image file should be named 'sample_nuclei.tif'.
   - Place all ROI image files in the same folder for streamlined access and analysis.



In [ ]:
# @title #MIT License

print("""
**MIT License**

Copyright (c) 2023 Guillaume Jacquemet

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.""")

--------------------------------------------------------
# **Part 0. Prepare the Google Colab session**
--------------------------------------------------------
<font size = 4>skip this section when using a local installation


## **0.1. Install key dependencies**
---
<font size = 4>

In [ ]:
#@markdown ##Play to install

print("In progress....")

!git clone https://github.com/CellMigrationLab/CellTracksColab.git

%pip -q install pandas scikit-learn
%pip -q install plotly
%pip -q install tqdm

## **0.2. Mount your Google Drive**
---
<font size = 4> To use this notebook on the data present in your Google Drive, you need to mount your Google Drive to this notebook.

<font size = 4> Play the cell below to mount your Google Drive and follow the instructions.

<font size = 4> Once this is done, your data are available in the **Files** tab on the top left of notebook.

In [ ]:
#@markdown ##Play the cell to connect your Google Drive to Colab
from google.colab import drive
drive.mount('/content/Gdrive')

--------------------------------------------------------
# **Part 1: Prepare the session and load your data**
--------------------------------------------------------


## **1.1. Load key dependencies**
---
<font size = 4>

In [ ]:
#@markdown ##Play to load the dependancies
import os
import pandas as pd
import seaborn as sns
import numpy as np
import sys
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import itertools
import requests
import ipywidgets as widgets
import warnings
import scipy.stats as stats
import gzip


from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.gridspec import GridSpec
from ipywidgets import Dropdown, interact,Layout, VBox, Button, Accordion, SelectMultiple, IntText
from tqdm.notebook import tqdm
from IPython.display import display, clear_output
from scipy.spatial import ConvexHull
from scipy.spatial.distance import cosine, pdist
from scipy.cluster.hierarchy import linkage, dendrogram
from sklearn.metrics import pairwise_distances
from scipy.stats import zscore, ks_2samp
from sklearn.preprocessing import MinMaxScaler
from multiprocessing import Pool
from matplotlib.ticker import FixedLocator
from matplotlib.ticker import FuncFormatter
from matplotlib.colors import LogNorm

sys.path.append("../")
sys.path.append("CellTracksColab/")

import celltracks
from celltracks import *
from celltracks.Track_Plots import *
from celltracks.BoxPlots_Statistics import *
from celltracks.Track_Metrics import *
from celltracks.Distance_to_ROI import *


# Current version of the notebook the user is running
current_version = "1.0.2"
Notebook_name = 'Distance_to_ROI'

# URL to the raw content of the version file in the repository
version_url = "https://raw.githubusercontent.com/guijacquemet/CellTracksColab/main/Notebook/latest_version.txt"

# Function to define colors for formatting messages
class bcolors:
    WARNING = '\033[91m'  # Red color for warning messages
    ENDC = '\033[0m'      # Reset color to default

# Check if this is the latest version of the notebook
try:
    All_notebook_versions = pd.read_csv(version_url, dtype=str)
    print('Notebook version: ' + current_version)

    # Check if 'Version' column exists in the DataFrame
    if 'Version' in All_notebook_versions.columns:
        Latest_Notebook_version = All_notebook_versions[All_notebook_versions["Notebook"] == Notebook_name]['Version'].iloc[0]
        print('Latest notebook version: ' + Latest_Notebook_version)

        if current_version == Latest_Notebook_version:
            print("This notebook is up-to-date.")
        else:
            print(bcolors.WARNING + "A new version of this notebook has been released. We recommend that you download it at https://github.com/guijacquemet/CellTracksColab" + bcolors.ENDC)
    else:
        print("The 'Version' column is not present in the version file.")
except requests.exceptions.RequestException as e:
    print("Unable to fetch the latest version information. Please check your internet connection.")
except Exception as e:
    print("An error occurred:", str(e))


## **1.2. Load existing CellTracksColab dataframes**
---

<font size = 4> Please ensure that your data was properly processed using CellTracksColab. To use the Viewer Notebook, your data must be formatted in the CellTracksColab format. This involves compiling your tracking data into two main DataFrames:

*   Your Track_table: `merged_tracks_df`

*   Spot_table: `merged_spots_df`.

<font size = 4>**Data_Dims**: Choose "2D" or "3D" for your data dimensions.

<font size = 4>**Results_Folder**: The directory path where the analysis results will be saved.

In [ ]:
#@markdown ##Provide the path to your CellTracksColab dataset:

Data_Dims = "2D" #@param ["2D", "3D"]
Data_Type = "CellTracksColab"

Track_table = ''  # @param {type: "string"}
Spot_table = ''  # @param {type: "string"}


Use_test_dataset = False

#@markdown ###Provide the path to your Result folder

Results_Folder = ""  # @param {type: "string"}

# Update the parameters to load the data
CellTracks = celltracks.TrackingData()
if Use_test_dataset:
    # Download the test dataset
    test_celltrackscolab = "https://zenodo.org/record/8420011/files/T_Cells_spots_only.zip?download=1"
    CellTracks.DownloadTestData(test_celltrackscolab)
    File_Format = "csv"
else:

    CellTracks.Spot_table = Spot_table
    CellTracks.Track_table = Track_table

CellTracks.Results_Folder = Results_Folder
CellTracks.skiprows = None
CellTracks.data_type = Data_Type
CellTracks.data_dims = Data_Dims

# Load data
CellTracks.LoadTrackingData()

merged_spots_df = CellTracks.spots_data
check_for_nans(merged_spots_df, "merged_spots_df")
merged_tracks_df = CellTracks.tracks_data
print("...Done")


## **1.3. Visualise your tracks**
---

In [ ]:
# @title ##Run the cell and choose the file you want to inspect
display_plots=True

if not os.path.exists(Results_Folder+"/Tracks"):
    os.makedirs(Results_Folder+"/Tracks")

filenames = merged_spots_df['File_name'].unique()

filename_dropdown = widgets.Dropdown(
    options=filenames,
    value=filenames[0] if len(filenames) > 0 else None,  # Default selected value
    description='File Name:',
)

interact(lambda filename: plot_track_coordinates(filename, merged_spots_df, Results_Folder, display_plots=display_plots), filename=filename_dropdown);


# **Part 2: Compute the distance to the nearest ROI**

<font size = 4>
This process can be repeated for different types of ROIs, such as 'Nuclei', 'Junctions', etc.
Simply change the `ROI_name` parameter to correspond to each specific ROI type you wish to analyze. For instance, set `ROI_name` to 'Nuclei' for analyzing nuclei, and then change it to 'Junctions' for junctions. This flexibility allows you to perform separate analyses for each ROI type using the same notebook, ensuring a thorough and comprehensive examination of tracks in relation to diverse ROI types.





## **2.1. Input Parameters**
<font size="4">

**ROI Folder Path**: Specify the directory containing your Region of Interest (ROI) images. These images should be masks or label images where the background has a pixel value of 0, and objects of interest are represented by values greater than 0.

**ROI Name**: Define the unique name for your region of interest (ROI). This name should align with the naming convention used in your image files. You can perform analyses on multiple ROIs one at a time by adjusting the `ROI_name` parameter for each specific ROI image. Image files should adhere to the following format: `File_name_ROI_name.tif`. For example, if your tracking file is named 'sample,' and the ROI is 'nuclei,' the corresponding image file should be named 'sample_nuclei.tif.'

**Pixel Calibration**: Enter the calibration value that converts pixel distances into real-world units (e.g., micrometers). This crucial parameter ensures accurate measurements in physical dimensions.




In [ ]:
ROI_folder = ''  # @param {type: "string"}

ROI_name = ''# @param {type: "string"}

Pixel_calibration = 0.6337449# @param {type: "number"}

if not os.path.exists(Results_Folder+"/Distance_to_ROI"):
    os.makedirs(Results_Folder+"/Distance_to_ROI")

if not os.path.exists(Results_Folder+"/Distance_to_ROI/"+ROI_name):
    os.makedirs(Results_Folder+"/Distance_to_ROI/"+ROI_name)

distance_column_name = f'DistanceTo{ROI_name}'

if distance_column_name in merged_spots_df.columns:
    print(f"The column '{distance_column_name}' already exists in the DataFrame.")
    print("This indicates that the distance to ROI calculation might have been done previously.")
    print("Computing the distance to ROI again may not required unless the data has changed.")

print(f"Done.")

## **2.2. Check that your dataset matches your image files**

1. **Checking and Correcting Coordinates**: This process assesses and corrects the coordinates in the dataset to ensure they fall within the bounds of the associated images or videos.

2. **Zero Pixel Percentage**: For each file in the dataset, the code calculates the percentage of background (zero) pixels in the associated Region of Interest (ROI) image or video. This metric helps evaluate the dataset's quality and whether the coordinates need adjustment.

3. **Correction Process**: If the coordinates are found outside the image or video bounds, the code corrects them, ensuring they are within acceptable limits. Any corrections made are reported in the console.

4. **Supported Data Types**: The code can handle image and video data. It determines the type and dimensions of the data automatically.

5. **Pixel Calibration**: The coordinates are adjusted for the specified pixel calibration, ensuring accurate positioning in real-world units.

By running this code, you validate and, if necessary, correct the coordinates in your dataset to match the dimensions of your ROI images or videos.


In [ ]:
# @title ##Check that your dataset matches your image files

from tqdm.notebook import tqdm
from tqdm.notebook import tqdm
import pandas as pd
from skimage import io
import matplotlib.pyplot as plt
from tifffile import imread
from skimage.measure import label, regionprops, find_contours
from scipy.ndimage import distance_transform_edt

# Apply the function to each file in the DataFrame
for file_name in tqdm(merged_spots_df['File_name'].unique(), desc="Checking and correcting coordinates"):
    merged_spots_df = check_and_correct_coordinates(merged_spots_df, file_name, ROI_folder, ROI_name, Pixel_calibration)

save_dataframe_with_progress(merged_spots_df, Results_Folder + '/' + 'merged_Spots.csv.gz', desc="Saving Spots")

## **2.3. Compute the distance to the nearest ROI**

By running this code, you compute and record the distances from each data point to the nearest ROI pixel, facilitating further analysis and interpretation of your dataset.




In [ ]:
# @title ##Compute the distance to the nearest ROI

from tqdm.notebook import tqdm
import pandas as pd
from skimage import io
import matplotlib.pyplot as plt
from tifffile import imread
from skimage.measure import label, regionprops, find_contours
from scipy.ndimage import distance_transform_edt


def compute_distances_using_distance_transform(df, image_dir, ROI_name, Pixel_calibration):
    """
    Compute distances to the nearest labeled pixel for each spot using the distance transform method.
    Automatically detects if the file is a single image or a video sequence and checks if the frame
    number corresponds to the actual number of frames in the video.

    Parameters:
    df (DataFrame): The dataframe containing the spots' data.
    image_dir (str): The directory where the ROI images or videos are stored.
    """
    for file_name in tqdm(df['File_name'].unique(), desc="Processing files"):
        # Paths to the label images or video
        ROI_img_path = f"{image_dir}/{file_name}_{ROI_name}.tif"

        try:
            ROI_img = io.imread(ROI_img_path)
            # Ensure the image dimensions are 3 or below
            if ROI_img.ndim > 3:
                raise ValueError(f"Image file {file_name} has more than 3 dimensions, which is not supported.")

            # Determine if the file is a video by checking if it has more than two dimensions
            is_video = ROI_img.ndim == 3

            # Verify that the 'FRAME' number in the dataframe does not exceed the number of frames in the video
            if is_video and 'FRAME' in df.columns:
                file_df = df[df['File_name'] == file_name]
            # Compute max_frame_num for the current file_name
                max_frame_num = file_df['FRAME'].max()
                num_frames = ROI_img.shape[0]
                if max_frame_num > num_frames:
                    print(f"Error: max_frame_num ({max_frame_num}) exceeds num_frames ({num_frames}) in file {file_name}.")
                    raise ValueError(f"DataFrame contains 'FRAME' numbers that exceed the number of frames in the video for file {file_name}.")
                for frame_idx in range(num_frames):
                    # Process each frame with matching spots
                    process_frame(ROI_img[frame_idx], df, file_name, frame_idx)
            else:
                # Process a single image
                process_frame(ROI_img, df, file_name)

        except FileNotFoundError:
            print(f"Error: Image for {file_name} not found. Skipping...")
            continue
        except ValueError as e:
            print(e)
            break

    return df

def process_frame(ROI_img, df, file_name, frame_idx=None):
    """
    Process a single frame or image and update the dataframe with distance values.

    Parameters:
    ROI_img (ndarray): The ROI image or a single frame from a video.
    df (DataFrame): The dataframe to update.
    file_name (str): The name of the file being processed.
    frame_idx (int, optional): The index of the frame in the video.
    """
    # Compute distance transform
    distance_transform_ROI = distance_transform_edt(ROI_img == 0) * Pixel_calibration

    # Filter dataframe for the current file and frame
    file_df = df[df['File_name'] == file_name]
    if frame_idx is not None:
        file_df = file_df[file_df['FRAME'] == frame_idx]

    for idx, row in tqdm(file_df.iterrows(), total=file_df.shape[0], desc=f"Processing coordinates for {file_name}", leave=False):
        y, x = int(row['POSITION_Y'] / Pixel_calibration), int(row['POSITION_X'] / Pixel_calibration)
                # Check if x and y are within the bounds of the image

        if 0 <= x < distance_transform_ROI.shape[1] and 0 <= y < distance_transform_ROI.shape[0]:
            df.loc[df.index[idx], f'DistanceTo{ROI_name}'] = distance_transform_ROI[y, x]
        else:
            print(f"Warning: Coordinates (x={x}, y={y}) out of bounds for {file_name}")


compute_distances_using_distance_transform(merged_spots_df, ROI_folder, ROI_name, Pixel_calibration)

save_dataframe_with_progress(merged_spots_df, Results_Folder + '/' + 'merged_Spots.csv.gz', desc="Saving Spots")

## **2.4. Check for missing values**

By running this code, you can quickly identify which filenames in your dataset contain data points with missing distance values. This information is valuable for data quality assessment. This likely occurs because ROI images are missing.



In [ ]:
# @title ##Check for NaN

print_filenames_with_nan_distances(merged_spots_df, ROI_name)


## **2.5. Visual validation**

1. **Filename Dropdown**: Select a specific filename from the dropdown list. This corresponds to the tracked data associated with the ROI you want to analyze.

2. **Display Option**: Choose between 'All' or 'Specific number' to control the number of points displayed on the plot.

3. **Number of Points**: If you select 'Specific number' in the display option, enter the desired number of points to display.

4. **Save as PDF**: Check this box if you want to save the generated plot as a PDF file.

5. **Visualize Distances Button**: Click this button to visualize the distances measured for the selected ROI and filename.

The resulting plot will display the ROI image with distances represented by red circles around the points of interest. Each circle's radius corresponds to the distance measurement in real-world units (e.g., micrometers), taking into account the specified pixel calibration.

If the 'Save as PDF' option is checked, the plot will be saved as a PDF file in the designated results folder.

Use this interface to visually inspect and validate the correctness of the measured distances for your ROI data.



In [ ]:
# @title ##Run to check visually that the distances measured are correct.
from ipywidgets import Button, interactive, IntSlider, widgets, fixed
from ipywidgets import Output
from IPython.display import clear_output
import numpy as np
import matplotlib.pyplot as plt
from tifffile import imread
import matplotlib.backends.backend_pdf


error_output = Output()

def display_error_message(message):
    with error_output:
        clear_output(wait=False)
        print(f"Error: {message}")

filename_dropdown = widgets.Dropdown(
    options=merged_spots_df['File_name'].unique(),
    description='Filename:',
    disabled=False
)

# Additional widgets for user input
display_option = widgets.Dropdown(
    options=['All', 'Specific number'],
    value='All',
    description='Display:',
    disabled=False
)

number_of_points = widgets.BoundedIntText(
    value=5,
    min=1,
    max=1,  # Adjust max as per your data range
    step=1,
    description='Number of points:',
    disabled=False
)

# Checkbox for saving plot as PDF
save_as_pdf_checkbox = widgets.Checkbox(
    value=False,
    description='Save as PDF',
    disabled=False
)

# Display these widgets immediately
display(filename_dropdown)

def update_display(frame_number, ROI_img, data_for_frame, filename, display_mode, point_count, save_pdf):
    plt.figure(figsize=(8, 8))

    frame = ROI_img.copy()
    if frame.ndim == 3:  # If the image is a video
        frame = frame[frame_number]

    coords_for_frame = data_for_frame[data_for_frame['FRAME'] == frame_number]
    if coords_for_frame.empty:
        print(f"No data available for frame {frame_number} in file {filename}.")
    else:
        plt.imshow(frame, cmap='gray')

        plt.xlim(0, frame.shape[1])
        plt.ylim(frame.shape[0], 0)

        if display_mode == 'All':
            points_to_display = coords_for_frame
        else:
            points_to_display = coords_for_frame.head(point_count)

        for idx, row in points_to_display.iterrows():
            x, y = int(row['POSITION_X']/Pixel_calibration), int(row['POSITION_Y']/Pixel_calibration)
            distance_to_edge = row[f'DistanceTo{ROI_name}']/Pixel_calibration
            circle_edge = plt.Circle((x, y), distance_to_edge, color='red', fill=False, linewidth=1)
            plt.gca().add_patch(circle_edge)

        plt.scatter(points_to_display['POSITION_X']/Pixel_calibration, points_to_display['POSITION_Y']/Pixel_calibration, c='yellow', s=50, zorder=3)


    plt.title(f"Frame {frame_number} for {filename}")

    if save_pdf:
        pdf_filename = f"{Results_Folder}/Distance_to_ROI/{ROI_name}/{filename}_frame_{frame_number}.pdf"
        plt.savefig(pdf_filename, format='pdf')
        print(f"Plot saved as '{pdf_filename}'")
    plt.show()

def visualize_precomputed_distances_for_filename(filename):
    ROI_img_path = f"{ROI_folder}/{filename}_{ROI_name}.tif"
        # Calculate the maximum number of points for the selected file
    max_points_for_file = merged_spots_df[merged_spots_df['File_name'] == filename].groupby('FRAME').size().max()

    # Update the maximum value for the number_of_points widget
    number_of_points.max = max_points_for_file

    try:
        ROI_img = imread(ROI_img_path)
    except FileNotFoundError:
        display_error_message(f"Image for {filename} not found.")
        return

    data_for_frame = merged_spots_df[merged_spots_df['File_name'] == filename]

    max_frame = data_for_frame['FRAME'].max()
    frame_slider = widgets.IntSlider(min=0, max=max_frame, description='Frame')

    # Modified call to include new widgets
    w = interactive(update_display,
                    frame_number=frame_slider,
                    ROI_img=fixed(ROI_img),
                    data_for_frame=fixed(data_for_frame),
                    filename=fixed(filename),
                    display_mode=display_option,
                    point_count=number_of_points,
                    save_pdf=save_as_pdf_checkbox)

    display(w)

# Button to trigger visualization
plot_button_filename = Button(description="Visualize Distances", button_style='info')

# Function to handle button click for filename visualization
def on_plot_button_filename_click(b):
    filename = filename_dropdown.value
    print("In progress...")
    # Clear the previous output
    clear_output()

    # Call the visualization function without redisplaying the widgets
    visualize_precomputed_distances_for_filename(filename)

# Bind the function to the button click event
plot_button_filename.on_click(on_plot_button_filename_click)

# Display the button and error output
display(plot_button_filename)
display(error_output)


## **2.6. Compute general track metrics associated with the ROI**


1. **MaxDistance_{ROI_name}**:
   - The maximum distance of the track from the ROI during the tracking period.
   - Indicates the farthest point reached relative to the ROI.

2. **MinDistance_{ROI_name}**:
   - The minimum distance of the track from the ROI during the tracking period.
   - Represents the closest approach to the ROI.

3. **StartDistance_{ROI_name}** and **EndDistance_{ROI_name}**:
   - Distances from the ROI at the start and end of the tracking period, respectively.
   - Useful for understanding initial and final positioning relative to the ROI.

4. **MedianDistance_{ROI_name}**:
   - The median of all recorded distances to the ROI.
   - Provides a central tendency measure, less affected by outliers than the mean.

5. **StdDevDistance_{ROI_name}**:
   - Standard deviation of the distances.
   - Indicates the variability or consistency of the track's distance from the ROI.

6. **DirectionMovement_{ROI_name}**:
   - Calculated as `EndDistance - StartDistance`.
   - A positive value indicates moving away from the ROI over time, and a negative value suggests moving closer.

7. **AvgRateChange_{ROI_name}**:
   - Average rate of change in distance per frame.
   - Helps assess the speed of movement towards or away from the ROI.

8. **PercentageChange_{ROI_name}**:
   - Percentage change in distance from the start to the end of the track.
   - Normalizes the movement relative to the initial distance.

9. **TrendSlope_{ROI_name}**:
   - Slope of a linear regression line fitted to the distance values over time.
   - Indicates the general trend of movement (increasing or decreasing distance).

<font size = 4>**Interpretation**
- These metrics are calculated considering the distance to the closest ROI at each time point. If the ROI is moving, the metrics reflect the relative motion between the track and the ROI.
- The closest ROI to a track at each time point may change if there are multiple ROIs. This factor is inherently considered in the distance calculations.
- It is essential to consider the movement of both the track and the ROI when interpreting these metrics. For example, a decreasing distance over time could mean the track is moving towards the ROI, the ROI is moving towards the track, or both.


In [ ]:
# @title ##Run to compute the metrics.

from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
from scipy.stats import linregress

# Using the function with your DataFrames
distances_metrics_df = get_distances_and_metrics(merged_tracks_df, merged_spots_df, ROI_name)

# Merging Process
overlapping_columns = merged_tracks_df.columns.intersection(distances_metrics_df.columns).drop('Unique_ID')
merged_tracks_df.drop(columns=overlapping_columns, inplace=True)
merged_tracks_df = pd.merge(merged_tracks_df, distances_metrics_df, on='Unique_ID', how='left')

# Save the updated DataFrame
save_dataframe_with_progress(merged_tracks_df, Results_Folder + '/' + 'merged_Tracks.csv.gz')

check_for_nans(merged_tracks_df, 'merged_tracks_df')


--------
# **Part 3. Quality Control**
--------

      



## **3.1. Assess if your dataset is balanced**
---

In cell tracking and similar biological analyses, the balance of the dataset is important, particularly in ensuring that each biological repeat carries equal weight. Here's why this balance is essential:

<font size = 4>**Accurate Representation of Biological Variability**

- **Capturing True Biological Variation**: Biological repeats are crucial for capturing the natural variability inherent in biological systems. Equal weighting ensures that this variability is accurately represented.
- **Reducing Sampling Bias**: By balancing the dataset, we avoid overemphasizing the characteristics of any single repeat, which might not be representative of the broader biological context.

If your data is too imbalanced, it may be useful to ensure that this does not shift your results.



In [ ]:
# @title ##Check the number of track per condition per repeats

if not os.path.exists(f"{Results_Folder}/QC"):
    os.makedirs(f"{Results_Folder}/QC")

result_df = count_tracks_by_condition_and_repeat(merged_tracks_df, f"{Results_Folder}/QC")


## **3.2. Compute Similarity Metrics between Field of Views (FOV) and between Conditions and Repeats**
---

<font size = 4>**Purpose**:

<font size = 4>This section provides a set of tools to compute and visualize similarities between different field of views (FOV) based on selected track parameters. By leveraging hierarchical clustering, the resulting dendrogram offers a clear visualization of how different FOV, conditions, or repeats relate to one another. This tool is essential for:

<font size = 4>1. **Quality Control**:
    - Ensuring that FOVs from the same condition or experimental setup are more similar to each other than to FOVs from different conditions.
    - Confirming that repeats of the same experiment yield consistent results and cluster together.
    
<font size = 4>2. **Data Integrity**:
    - Identifying potential outliers or anomalies in the dataset.
    - Assessing the overall consistency of the experiment and ensuring reproducibility.

<font size = 4>**How to Use**:

<font size = 4>1. **Track Parameters Selection**:
    - A list of checkboxes allows users to select which track parameters they want to consider for similarity calculations. By default, all parameters are selected. Users can deselect parameters that they believe might not contribute significantly to the similarity.

<font size = 4>2. **Similarity Metric**:
    - Users can choose a similarity metric from a dropdown list. Options include cosine, euclidean, cityblock, jaccard, and correlation. The choice of similarity metric can influence the clustering results, so users might need to experiment with different metrics to see which one provides the most meaningful results.

<font size = 4>3. **Linkage Method**:
    - Determines how the distance between clusters is calculated in the hierarchical clustering process. Different linkage methods can produce different dendrograms, so users might want to try various methods.

<font size = 4>4. **Visualization**:
    - Once the parameters are selected, users can click on the "Select the track parameters and visualize similarity" button. This will compute the hierarchical clustering and display two dendrograms:
        - One dendrogram displays similarities between individual FOVs.
        - Another dendrogram aggregates the data based on conditions and repeats, providing a higher-level view of the similarities.
      


In [ ]:
# @title ##Compute similarity metrics between FOV and between conditions and repeats

# Check and create "QC" folder
if not os.path.exists(f"{Results_Folder}/QC"):
    os.makedirs(f"{Results_Folder}/QC")

# Columns to exclude
excluded_columns = ['Condition', 'experiment_nb', 'File_name', 'Repeat', 'Unique_ID', 'LABEL', 'TRACK_INDEX', 'TRACK_ID', 'TRACK_X_LOCATION', 'TRACK_Y_LOCATION', 'TRACK_Z_LOCATION', 'Exemplar','TRACK_STOP', 'TRACK_START', 'Cluster_UMAP', 'Cluster_tsne']

selected_df = pd.DataFrame()

# Filter out non-numeric columns but keep 'File_name'
numeric_df = merged_tracks_df.select_dtypes(include=['float64', 'int64']).copy()
numeric_df['File_name'] = merged_tracks_df['File_name']

# Create a list of column names excluding 'File_name'
column_names = [col for col in numeric_df.columns if col not in excluded_columns]

# Create a checkbox for each column
checkboxes = [widgets.Checkbox(value=True, description=col, indent=False) for col in column_names]

# Dropdown for similarity metrics
similarity_dropdown = widgets.Dropdown(
    options=['cosine', 'euclidean', 'cityblock', 'jaccard', 'correlation'],
    value='cosine',
    description='Similarity Metric:'
)

# Dropdown for linkage methods
linkage_dropdown = widgets.Dropdown(
    options=['single', 'complete', 'average', 'ward'],
    value='single',
    description='Linkage Method:'
)

# Arrange checkboxes in a 2x grid
grid = widgets.GridBox(checkboxes, layout=widgets.Layout(grid_template_columns="repeat(2, 300px)"))

# Create a button to trigger the selection and visualization
button = widgets.Button(description="Select the track parameters and visualize similarity", layout=widgets.Layout(width='400px'), button_style='info')

# Define the button click event handler
def on_button_click(b):
    global selected_df  # Declare selected_df as global

    # Get the selected columns from the checkboxes
    selected_columns = [box.description for box in checkboxes if box.value]
    selected_columns.append('File_name')  # Always include 'File_name'

    # Extract the selected columns from the DataFrame
    selected_df = numeric_df[selected_columns]

    # Check and print the percentage of NaNs for each selected column
    for column in selected_columns:
        if selected_df[column].isna().any():
            nan_percentage = selected_df[column].isna().mean() * 100
            print("Warning: NaN values found in the selected data.")
            print(f"{column}: {nan_percentage:.2f}%")
            any_nan = True
            print("Proceeding to handle NaN values.")
            selected_df = selected_df.dropna()

    # Aggregate the data by filename
    aggregated_by_filename = selected_df.groupby('File_name').mean(numeric_only=True)

    # Aggregate the data by condition and repeat
    aggregated_by_condition_repeat = merged_tracks_df.groupby(['Condition', 'Repeat'])[selected_columns].mean(numeric_only=True)

    # Compute condensed distance matrices
    distance_matrix_filename = pdist(aggregated_by_filename, metric=similarity_dropdown.value)
    distance_matrix_condition_repeat = pdist(aggregated_by_condition_repeat, metric=similarity_dropdown.value)

    # Perform hierarchical clustering
    linked_filename = linkage(distance_matrix_filename, method=linkage_dropdown.value)
    linked_condition_repeat = linkage(distance_matrix_condition_repeat, method=linkage_dropdown.value)

    annotation_text = f"Similarity Method: {similarity_dropdown.value}, Linkage Method: {linkage_dropdown.value}"

        # Prepare the parameters dictionary
    similarity_params = {
        'Similarity Metric': similarity_dropdown.value,
        'Linkage Method': linkage_dropdown.value,
        'Selected Columns': ', '.join(selected_columns)
    }

    # Save the parameters
    params_file_path = os.path.join(Results_Folder, "QC/analysis_parameters.csv")
    save_parameters(similarity_params, params_file_path, 'Similarity Metrics')

    # Plot the dendrograms one under the other
    plt.figure(figsize=(10, 10))

    # Dendrogram for individual filenames
    plt.subplot(2, 1, 1)
    dendrogram(linked_filename, labels=aggregated_by_filename.index, orientation='top', distance_sort='descending', leaf_rotation=90)
    plt.title(f'Dendrogram of Field of view Similarities\n{annotation_text}')

    # Dendrogram for aggregated data based on condition and repeat
    plt.subplot(2, 1, 2)
    dendrogram(linked_condition_repeat, labels=aggregated_by_condition_repeat.index, orientation='top', distance_sort='descending', leaf_rotation=90)
    plt.title(f'Dendrogram of Aggregated Similarities by Condition and Repeat\n{annotation_text}')

    plt.tight_layout()

    # Save the dendrogram to a PDF
    pdf_pages = PdfPages(f"{Results_Folder}/QC/Dendrogram_Similarities.pdf")

    # Save the current figure to the PDF
    pdf_pages.savefig()

    # Close the PdfPages object to finalize the document
    pdf_pages.close()

    plt.show()

# Set the button click event handler
button.on_click(on_button_click)

# Display the widgets
display(grid, similarity_dropdown, linkage_dropdown, button)


-------------------------------------------

# **Part 4. Plot track parameters**
-------------------------------------------

<font size="4" color="red">
<b>Results Storage:</b>
Results generated by in this section are saved  in the sub-folder named `track_parameters_plots` within your `Results_Folder`.


<font size="4"> In this section, you can plot all the track metrics previously computed. Data and graphs are automatically saved in your result folder.

<font size="4" color="red"> Parameters computed are in the unit you provided when tracking your data.

### **Track Metrics Available**

<font size="4">The metrics can be computed in the previous section of the notebook, in other CellTracksColab notebooks, or imported directly from the tracking software. To make the selection process user-friendly, the metrics are categorized as follows:

<font size="4">1. **Track Metrics**: Includes fundamental metrics such as Track Duration, Mean Speed, Median Speed, Max Speed, Min Speed, Speed Standard Deviation, Total Distance Traveled, Spatial Coverage, Tortuosity, and Total Turning Angle.

<font size="4">2. **Rolling Track Metrics**: Calculated over a rolling window, including Mean Speed Rolling, Median Speed Rolling, Max Speed Rolling, Min Speed Rolling, Speed Standard Deviation Rolling, Total Distance Traveled Rolling, Directionality Rolling, Tortuosity Rolling, Total Turning Angle Rolling, and Spatial Coverage Rolling.

<font size="4">3. **Morphological Metrics**: Metrics related to shape and size (when available).

<font size="4">4. **Distance to ROI Metrics**: Calculated relative to regions of interest and computed in the CellTracksColab distance to ROI notebook (ROIs).

<font size="4">In addition to metrics computed within CellTracksColab, we import metrics computed directly by the tracking software.

<font size="4">These metrics are organized into an expandable and collapsible accordion menu grouped by the categories above. Each category can be individually expanded or collapsed, and all sections are closed by default. A "Select All" checkbox is provided for each category, allowing users to select or deselect all metrics within a category quickly.

<font size="4">Learn more about the parameters available on our [wiki](https://github.com/CellMigrationLab/CellTracksColab/wiki/Track-Metrics).

### **Statistical Analyses**

<font size = 4>**Cohen's d (Effect Size)**:

<font size="4">Cohen's d measures the size of the difference between two groups, normalized by their pooled standard deviation. Values can be interpreted as small (0 to 0.2), medium (0.2 to 0.5), or large (0.5 and above) effects. It helps quantify how significant the observed difference is, beyond just being statistically significant.

<font size = 4>**Randomization Test**:

<font size="4">This non-parametric test evaluates if observed differences between conditions could have arisen by random chance. It shuffles condition labels multiple times, recalculating Cohen's d each time. The resulting p-value, which indicates the likelihood of observing the actual difference by chance, provides evidence against the null hypothesis: a smaller p-value implies stronger evidence against the null.

<font size = 4>**t-tests Calculating P-Values Based on the Means of Repeats**:

<font size="4">This statistical test compares the means of different conditions to determine if they are statistically different. The t-test calculates p-values based on the means of the repeats, as described in the SuperPlots methodology.

<font size = 4>**Bonferroni Correction**:

<font size="4">Given multiple comparisons, the Bonferroni Correction adjusts significance thresholds to mitigate the risk of false positives. By dividing the standard significance level (alpha) by the number of tests, it ensures that only robust findings are considered significant. However, it's worth noting that this method can be conservative, sometimes overlooking genuine effects.

### **Choosing Between Randomization Test and T-Tests**

- <font size="4">**Randomization Test**:
  - <font size="4">**Advantages**: Non-parametric, does not assume normal distribution.
  - <font size="4">**Disadvantages**: Computationally intensive, especially with a large number of conditions.
  - <font size="4">**Best Use**: When you have a small number of repeats or suspect that your data may not follow a normal distribution.

- <font size="4">**t-tests**:
  - <font size="4">**Advantages**: Faster computation, widely understood and used.
  - <font size="4">**Disadvantages**: Assumes normal distribution of data.
  - <font size="4">**Best Use**: When you have a larger number of repeats and believe that your data follows a normal distribution.

## **4.1. Plot your entire dataset**
--------

In [ ]:
# @title ##Plot track normalized track parameters based on conditions as an heatmap (entire dataset)

base_folder = f"{Results_Folder}/track_parameters_plots"
Conditions = 'Condition'
df_to_plot = merged_tracks_df

folders = ["pdf", "csv"]
for folder in folders:
    dir_path = os.path.join(base_folder, folder)
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

# Example usage
heatmap_comparison(merged_tracks_df, base_folder, Conditions, normalization="zscore")

In [ ]:
# @title ##Plot track parameters (entire dataset)

base_folder = f"{Results_Folder}/track_parameters_plots"
Conditions = 'Condition'
df_to_plot = merged_tracks_df

folders = ["pdf", "csv"]
for folder in folders:
    dir_path = os.path.join(base_folder, folder)
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

condition_selector, condition_accordion = display_condition_selection(df_to_plot, Conditions)
checkboxes_dict, checkboxes_accordion = display_variable_checkboxes(categorize_columns(df_to_plot))
variable_checkboxes, checkboxes_widget = display_variable_checkboxes(get_selectable_columns_plots(df_to_plot))
stat_method_selector = widgets.Dropdown(
    options=['randomization test', 't-test'],
    value='randomization test',
    description='Stat Method:',
    style={'description_width': 'initial'}
)

button = Button(description="Plot Selected Variables", layout=Layout(width='400px'), button_style='info')
button.on_click(lambda b: plot_selected_vars(b, checkboxes_dict, df_to_plot, Conditions, base_folder, condition_selector, stat_method_selector));

display(VBox([condition_accordion, checkboxes_accordion, stat_method_selector, button]))

## **4.2. Plot a balanced dataset**
--------

## **4.2.1. Downsample your dataset to ensure that it is balanced**
--------

<font size = 4>**Downsampling and Balancing Dataset**

<font size = 4>This section of the notebook is dedicated to addressing imbalances in the dataset, which is crucial for ensuring the accuracy and reliability of the analysis. The cell bellow will downsample the dataset to balance the number of tracks across different conditions and repeats. It allows for reproducibility by including a `random_seed` parameter, which is set to 42 by default but can be adjusted as needed.

<font size = 4>All results from this section will be saved in the Balanced Dataset Directory created in your `Results_Folder`.




In [ ]:
# @title ##Run this cell to downsample and balance your dataset

random_seed = 42

if not os.path.exists(f"{Results_Folder}/Balanced_dataset"):
    os.makedirs(f"{Results_Folder}/Balanced_dataset")

balanced_merged_tracks_df = balance_dataset(merged_tracks_df, random_seed=random_seed)
result_df = count_tracks_by_condition_and_repeat(balanced_merged_tracks_df, f"{Results_Folder}/Balanced_dataset")

check_for_nans(balanced_merged_tracks_df, "balanced_merged_tracks_df")
save_dataframe_with_progress(balanced_merged_tracks_df, Results_Folder + '/Balanced_dataset/merged_Tracks_balanced_dataset.csv.gz')


## **4.2.2. Check if the downsampling has affected data distribution**
--------

<font size = 4>This section of the notebook generates a heatmap visualizing the Kolmogorov-Smirnov (KS) p-values for each numerical column in the dataset, comparing the distributions before and after downsampling. This heatmap serves as a tool for assessing the impact of downsampling on data quality, guiding decisions on whether the downsampled dataset is suitable for further analysis.

<font size = 4>**Purpose of the Heatmap**

- <font size = 4>**KS Test:** The KS test is used to determine if two samples are drawn from the same distribution. In this context, it compares the distribution of each numerical column in the original dataset (`merged_tracks_df`) with its counterpart in the downsampled dataset (`balanced_merged_tracks_df`).
- <font size = 4>**P-Value Interpretation:** The p-value indicates the probability that the two samples come from the same distribution. A higher p-value suggests a greater likelihood that the distributions are similar.

<font size = 4>**Interpreting the Heatmap**

- <font size = 4>**Color Coding:** The heatmap uses a color gradient (from viridis) to represent the range of p-values. Darker colors indicate higher p-values.
- <font size = 4>**P-Value Thresholds:**
  - <font size = 4>**High P-Values (Lighter Areas):** Indicate that the downsampling process likely did not significantly alter the distribution of that numerical column for the specific condition-repeat group.
  - <font size = 4>**Low P-Values (Darker Areas):** Suggest that the downsampling process may have affected the distribution significantly.
- <font size = 4>**Varying P-Values:** Variations in color across different columns and rows help identify which specific numerical columns and condition-repeat groups are most affected by the downsampling.




In [ ]:
# @title ##Check if your downsampling has affected your data distribution

numerical_columns = merged_tracks_df.select_dtypes(include=['int64', 'float64']).columns

# Initialize a DataFrame to store KS p-values
ks_p_values = pd.DataFrame(columns=numerical_columns)

# Iterate over each group and numerical column
for group, group_df in merged_tracks_df.groupby(['Condition', 'Repeat']):
    group_p_values = []
    balanced_group_df = balanced_merged_tracks_df[(balanced_merged_tracks_df['Condition'] == group[0]) & (balanced_merged_tracks_df['Repeat'] == group[1])]
    for column in numerical_columns:
        p_value = calculate_ks_p_value(group_df, balanced_group_df, column)
        group_p_values.append(p_value)
    ks_p_values.loc[f'Condition: {group[0]}, Repeat: {group[1]}'] = group_p_values

max_columns_per_heatmap = 20

total_columns = len(ks_p_values.columns)

num_heatmaps = -(-total_columns // max_columns_per_heatmap)  # Ceiling division

pdf_filepath = Results_Folder+'/Balanced_dataset/p-Value Heatmap.pdf'

# Create a PDF file
with PdfPages(pdf_filepath) as pdf:
    # Loop through each subset of columns and create a heatmap
    for i in range(num_heatmaps):
        start_col = i * max_columns_per_heatmap
        end_col = min(start_col + max_columns_per_heatmap, total_columns)

        # Subset of columns for this heatmap
        subset_columns = ks_p_values.columns[start_col:end_col]

        # Create the heatmap for the subset of columns
        plt.figure(figsize=(12, 8))
        sns.heatmap(ks_p_values[subset_columns], cmap='viridis', vmax=0.5, vmin=0)
        plt.title(f'Kolmogorov-Smirnov P-Value Heatmap (Columns {start_col+1} to {end_col})')
        plt.xlabel('Numerical Columns')
        plt.ylabel('Condition-Repeat Groups')
        plt.tight_layout()

        # Save the current figure to the PDF
        pdf.savefig()
        plt.show()
        plt.close()

print(f"Saved all heatmaps to {pdf_filepath}")

ks_p_values.to_csv(Results_Folder + '/Balanced_dataset/ks_p_values.csv')
print("Saved KS p-values to ks_p_values.csv")


## **4.2.3. Plot your balanced dataset**
--------

In [ ]:
# @title ##Plot track parameters (balanced dataset)

# Parameters to adapt in function of the notebook section
base_folder = f"{Results_Folder}/Balanced_dataset/track_parameters_plots"
Conditions = 'Condition'
df_to_plot = balanced_merged_tracks_df

# Check and create necessary directories
folders = ["pdf", "csv"]
for folder in folders:
    dir_path = os.path.join(base_folder, folder)
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

condition_selector, condition_accordion = display_condition_selection(df_to_plot, Conditions)
checkboxes_dict, checkboxes_accordion = display_variable_checkboxes(categorize_columns(df_to_plot))
variable_checkboxes, checkboxes_widget = display_variable_checkboxes(get_selectable_columns_plots(df_to_plot))
stat_method_selector = widgets.Dropdown(
    options=['randomization test', 't-test'],
    value='randomization test',
    description='Stat Method:',
    style={'description_width': 'initial'}
)

button = Button(description="Plot Selected Variables", layout=Layout(width='400px'), button_style='info')
button.on_click(lambda b: plot_selected_vars(b, checkboxes_dict, df_to_plot, Conditions, base_folder, condition_selector, stat_method_selector));

display(VBox([condition_accordion, checkboxes_accordion, stat_method_selector, button]))

# **Part 5. Classify your tracks by distance and plot your dataset**
--------
<font size = 4>
<b>Note on Units:</b> The parameters plotted are in the unit of measurement you used when tracking your data.
</font>


<font size="4" color="red">
<b>Results Storage:</b> The results generated in this section are saved in the sub-folder named `Distance_to_ROI` located within your `Results_Folder`.


In [ ]:
# @title ##Check the distribution of your dataset

ROI_name = 'edge'  #@param {type:"string"}

# Example usage
plot_tracks_vs_distance(merged_tracks_df, ROI_name)  # Replace 'ROI_name' with the actual ROI name


## **5.1. Classify your tracks by distance**
--------
This section classify tracks based on distance from a specified Region of Interest (ROI).

<font size = 4>**Input Parameters**
1. `distance_threshold` (type: number): The distance in units (as per your data) that defines the threshold for classification. Tracks within this distance (based on the Max distance computed previously) from the ROI are classified as 'Close,' and those beyond it as 'Far.'
2. `ROI_name` (type: string): The name of the Region of Interest. This is used to label the classification columns in the DataFrame.

In [ ]:
distance_threshold = 50  #@param {type:"number"}
ROI_name = ''  #@param {type:"string"}

# Classifying tracks
classified_tracks_df = classify_tracks_by_distance(merged_tracks_df, distance_threshold, ROI_name)

# Displaying and saving results
close_tracks_count = len(classified_tracks_df[classified_tracks_df[f'Track_Classification_{ROI_name}'] == f'Close_{ROI_name}'])
far_tracks_count = len(classified_tracks_df) - close_tracks_count

print(f"Classification of tracks based on distance to {ROI_name}:")
print(f"Close to {ROI_name}: {close_tracks_count}")
print(f"Far from {ROI_name}: {far_tracks_count}")

save_dataframe_with_progress(merged_tracks_df, Results_Folder + '/' + 'merged_Tracks.csv.gz')

## **5.2. Plot your entire dataset**
--------

In [ ]:
# @title ##Plot track normalized track parameters based on conditions as an heatmap (entire dataset)

base_folder = f"{Results_Folder}/Distance_to_ROI/{ROI_name}/Classified_track_parameters_plots"
Conditions = f'Track_Classification_Condition_{ROI_name}'
df_to_plot = merged_tracks_df

folders = ["pdf", "csv"]
for folder in folders:
    dir_path = os.path.join(base_folder, folder)
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

# Example usage
heatmap_comparison(merged_tracks_df, base_folder, Conditions, normalization="zscore")

In [ ]:
# @title ##Plot track parameters (entire dataset)

base_folder = f"{Results_Folder}/Distance_to_ROI/{ROI_name}/Classified_track_parameters_plots"
Conditions = f'Track_Classification_Condition_{ROI_name}'
df_to_plot = merged_tracks_df

folders = ["pdf", "csv"]
for folder in folders:
    dir_path = os.path.join(base_folder, folder)
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

condition_selector, condition_accordion = display_condition_selection(df_to_plot, Conditions)
checkboxes_dict, checkboxes_accordion = display_variable_checkboxes(categorize_columns(df_to_plot))
variable_checkboxes, checkboxes_widget = display_variable_checkboxes(get_selectable_columns_plots(df_to_plot))
stat_method_selector = widgets.Dropdown(
    options=['randomization test', 't-test'],
    value='randomization test',
    description='Stat Method:',
    style={'description_width': 'initial'}
)

button = Button(description="Plot Selected Variables", layout=Layout(width='400px'), button_style='info')
button.on_click(lambda b: plot_selected_vars(b, checkboxes_dict, df_to_plot, Conditions, base_folder, condition_selector, stat_method_selector));

display(VBox([condition_accordion, checkboxes_accordion, stat_method_selector, button]))

## **5.3. Plot a balanced dataset**
--------

## **5.3.1. Downsample your dataset to ensure that it is balanced**
--------

<font size = 4>**Downsampling and Balancing Dataset**

This section of the notebook is dedicated to addressing imbalances in the dataset, which is crucial for ensuring the accuracy and reliability of the analysis. The cell bellow will downsample the dataset to balance the number of tracks across different conditions and repeats. It allows for reproducibility by including a `random_seed` parameter, which is set to 42 by default but can be adjusted as needed.


<font size="4" color="red">
<b>Results Storage:</b> The results generated in this section are saved in the sub-folder named `Distance_to_ROI` located within your `Results_Folder`.



In [ ]:
# @title ##Run this cell to downsample and balance your dataset

random_seed = 42

if not os.path.exists(f"{Results_Folder}/Distance_to_ROI/{ROI_name}/Classified_Balanced_dataset"):
    os.makedirs(f"{Results_Folder}/Distance_to_ROI/{ROI_name}/Classified_Balanced_dataset")

balanced_merged_tracks_df = balance_dataset(merged_tracks_df, random_seed=random_seed)
result_df = count_tracks_by_condition_and_repeat(balanced_merged_tracks_df, f"{Results_Folder}/Distance_to_ROI/{ROI_name}/Classified_Balanced_dataset")

check_for_nans(balanced_merged_tracks_df, "balanced_merged_tracks_df")
save_dataframe_with_progress(balanced_merged_tracks_df,f"{Results_Folder}/Distance_to_ROI/{ROI_name}/Classified_Balanced_dataset/merged_Tracks_balanced_dataset.csv.gz")


## **5.3.2. Check if the downsampling has affected data distribution**
--------

This section of the notebook generates a heatmap visualizing the Kolmogorov-Smirnov (KS) p-values for each numerical column in the dataset, comparing the distributions before and after downsampling. This heatmap serves as a tool for assessing the impact of downsampling on data quality, guiding decisions on whether the downsampled dataset is suitable for further analysis.

<font size = 4>**Purpose of the Heatmap**
- **KS Test:** The KS test is used to determine if two samples are drawn from the same distribution. In this context, it compares the distribution of each numerical column in the original dataset (`merged_tracks_df`) with its counterpart in the downsampled dataset (`balanced_merged_tracks_df`).
- **P-Value Interpretation:** The p-value indicates the probability that the two samples come from the same distribution. A higher p-value suggests a greater likelihood that the distributions are similar.

<font size = 4>**Interpreting the Heatmap**
- **Color Coding:** The heatmap uses a color gradient (from viridis) to represent the range of p-values. Darker colors indicate higher p-values.
- **P-Value Thresholds:**
  - **High P-Values (Lighter Areas):** Indicate that the downsampling process likely did not significantly alter the distribution of that numerical column for the specific condition-repeat group.
  - **Low P-Values (Darker Areas):** Suggest that the downsampling process may have affected the distribution significantly.
- **Varying P-Values:** Variations in color across different columns and rows help identify which specific numerical columns and condition-repeat groups are most affected by the downsampling.




In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ks_2samp

# @title ##Check if your downsampling has affected your data distribution

# Identify numerical columns
numerical_columns = merged_tracks_df.select_dtypes(include=['int64', 'float64']).columns

# Initialize a DataFrame to store KS p-values
ks_p_values = pd.DataFrame(columns=numerical_columns)

# Iterate over each group and numerical column
for group, group_df in merged_tracks_df.groupby([f'Track_Classification_Condition_{ROI_name}', 'Repeat']):
    group_p_values = []
    balanced_group_df = balanced_merged_tracks_df[(balanced_merged_tracks_df[f'Track_Classification_Condition_{ROI_name}'] == group[0]) & (balanced_merged_tracks_df['Repeat'] == group[1])]
    for column in numerical_columns:
        p_value = calculate_ks_p_value(group_df, balanced_group_df, column)
        group_p_values.append(p_value)
    ks_p_values.loc[f'{group[0]}, R: {group[1]}'] = group_p_values

# Maximum number of columns per heatmap
max_columns_per_heatmap = 20

# Total number of columns
total_columns = len(ks_p_values.columns)

# Calculate the number of heatmaps needed
num_heatmaps = -(-total_columns // max_columns_per_heatmap)  # Ceiling division

# File path for the PDF
pdf_filepath = f"{Results_Folder}/Distance_to_ROI/{ROI_name}/Classified_Balanced_dataset/KS_p_values_heatmaps.pdf"

# Create a PDF file
with PdfPages(pdf_filepath) as pdf:
    # Loop through each subset of columns and create a heatmap
    for i in range(num_heatmaps):
        start_col = i * max_columns_per_heatmap
        end_col = min(start_col + max_columns_per_heatmap, total_columns)

        # Subset of columns for this heatmap
        subset_columns = ks_p_values.columns[start_col:end_col]

        # Create the heatmap for the subset of columns
        plt.figure(figsize=(12, 8))
        sns.heatmap(ks_p_values[subset_columns], cmap='viridis', vmax=0.5, vmin=0)
        plt.title(f'Kolmogorov-Smirnov P-Value Heatmap (Columns {start_col+1} to {end_col})')
        plt.xlabel('Numerical Columns')
        plt.ylabel('Condition-Repeat Groups')
        plt.tight_layout()

        # Save the current figure to the PDF
        pdf.savefig()
        plt.show()
        plt.close()

print(f"Saved all heatmaps to {pdf_filepath}")

# Save the p-values to a CSV file
ks_p_values.to_csv(f"{Results_Folder}/Distance_to_ROI/{ROI_name}/Classified_Balanced_dataset/ks_p_values.csv")
print("Saved KS p-values to ks_p_values.csv")


## **5.3.3. Plot your balanced dataset**
--------

In [ ]:
# @title ##Plot track parameters (balanced dataset)

# Parameters to adapt in function of the notebook section
base_folder = f"{Results_Folder}/Distance_to_ROI/{ROI_name}/Classified_Balanced_dataset/Classified_track_parameters_plots"
Conditions = f'Track_Classification_Condition_{ROI_name}'
df_to_plot = balanced_merged_tracks_df

# Check and create necessary directories
folders = ["pdf", "csv"]
for folder in folders:
    dir_path = os.path.join(base_folder, folder)
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

condition_selector, condition_accordion = display_condition_selection(df_to_plot, Conditions)
checkboxes_dict, checkboxes_accordion = display_variable_checkboxes(categorize_columns(df_to_plot))
variable_checkboxes, checkboxes_widget = display_variable_checkboxes(get_selectable_columns_plots(df_to_plot))
stat_method_selector = widgets.Dropdown(
    options=['randomization test', 't-test'],
    value='randomization test',
    description='Stat Method:',
    style={'description_width': 'initial'}
)

button = Button(description="Plot Selected Variables", layout=Layout(width='400px'), button_style='info')
button.on_click(lambda b: plot_selected_vars(b, checkboxes_dict, df_to_plot, Conditions, base_folder, condition_selector, stat_method_selector));

display(VBox([condition_accordion, checkboxes_accordion, stat_method_selector, button]))

# **Part 6: Version log**
---
<font size = 4>While I strive to provide accurate and helpful information, please be aware that:
  - This notebook may contain bugs.
  - Features are currently limited and will be expanded in future releases.

<font size = 4>We encourage users to report any issues or suggestions for improvement. Please check the [repository](https://github.com/guijacquemet/CellTracksColab) regularly for updates and the latest version of this notebook.


<font size = 4>**Version 1.0.1**
  - Includes a general data reader
  - Plotting functions are imported from the main code

<font size = 4>**Version 0.9.1**
  - Added the PIP freeze option to save a requirement text
  - Added the heatmap visualisation of track parameters
  - Heatmaps can now be displayed on multiple pages
  - Fix userwarning message during plotting (all box plots)

<font size = 4>**Version 0.9**
  - Improved plotting strategy. Specific conditions can be chosen
  - absolute cohen d values are now shown
  - In the QC the heatmap is automatically divided in subplot when too many columns are in the df

<font size = 4>**Version 0.8**
  - Settings are now saved
  - Order of the section has been modified to help streamline biological discoveries
  - New section added to quality Control to check if the dataset is balanced

<font size = 4>**Version 0.7**
  - First release of this notebook

